In [1]:
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from imutils import paths

from subprocess import check_output

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.applications.densenet import preprocess_input
#what

C:\Users\jdu12\Anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# load a single image to np array
def get_image(img_path):
    img = load_img(img_path, target_size=(299, 299))
    img = img_to_array(img)
    img = img/255
    return img

def to_one_hot(labels):
    id2class = dict()
    class2id = dict()      ##!!! class(integer) to id(whale class)
    num_labels = []
    counter = 0
    for id in labels:
        if id not in id2class.keys():
            num_labels.append(counter)
            id2class[id] = counter
            class2id[counter] = id
            counter += 1
        else:
            num_labels.append(id2class[id])
    one_hot_labels = to_categorical(num_labels, num_classes = 4251)
    return one_hot_labels, class2id, id2class, num_labels


# define data generator
#use of an image generator for preprocessing and data augmentation
image_gen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    #rescale=1./255,
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True
)


df = pd.read_csv("C:/Users/jdu12/Desktop/humpback/train.csv")

labels = df['Id']
labels, class2id, id2class, num_labels = to_one_hot(labels)

# generate class weights
from sklearn.utils import class_weight
cw = class_weight.compute_class_weight('balanced', np.arange(len(class2id)), np.asarray(num_labels))
cw = dict(enumerate(cw.flatten(), 1))
# since cw start at 1 instead of 0, we need to offset this by 1
cw_z = dict()
for key in cw:
    cw_z[key-1] = cw[key]

In [3]:
# loading train images
trainPaths = list(paths.list_images('C:/Users/jdu12/Desktop/humpback/train/'))
print(len(trainPaths))
train = np.array([get_image(img_path) for img_path in trainPaths])
print(train.shape)

9082
(9082, 299, 299, 3)


In [6]:
# split the data into train and val
# The dataset is super unbalanced, as there are many classes that contains only 1 image
# As a result, train/val data cannot be split before generating more data by augmentation
train_dir = 'C:/Users/jdu12/Desktop/humpback/train/'

num_categories = len(df['Id'].unique())
#print(num_categories)
validation = np.zeros((num_categories, 224, 224, 3))
validation_y = []

i = 0
for id in df['Id'].unique():
    validation_y.append(id)
    im = df[df['Id'] == id].sample(1)
    name =  np.array(im.get('Image'))[0]
    im = get_image(train_dir + name)

    # https://www.kaggle.com/lextoumbourou/humpback-whale-id-data-and-aug-exploration
    x = random.randint(0, 3)
    if x == 0:
        validation[i,:, :, :] = random_rotation(im, 30, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    if x == 1:
        validation[i,:, :, :] = random_shift(im, wrg=0.1, hrg=0.3, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    if x == 2:
        validation[i,:, :, :] = random_shear(im, intensity=0.4, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    if x == 3:
        validation[i,:, :, :] = random_zoom(im, zoom_range=(1.5, 0.7), row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    i = i + 1

validation_y = to_one_hot(validation_y)[0]

In [5]:
# acquire the model
model = keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=True, weights=None, classes=4251)

In [9]:
batch_size = 32
epochs = 200

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_dir = 'C:/Users/jdu12/Desktop/humpback/saved_model/InceptionResnetV2/'
routine_dir = model_dir + "routine-{epoch:02d}-{acc:.2f}.hdf5"
routine_save = ModelCheckpoint(routine_dir, monitor='acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=10)
best_dir = model_dir + "best-{epoch:02d}-{acc:.2f}.hdf5"
best_save = ModelCheckpoint(best_dir, monitor='acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=3)


hist = model.fit_generator(image_gen.flow(train, labels, batch_size=batch_size),
                           steps_per_epoch=  train.shape[0]//batch_size,
                           epochs=epochs, verbose=1, callbacks=[routine_save, best_save], class_weight=cw_z,
                           validation_data=(validation, validation_y))


Epoch 1/180
283/283 [==============================] - 249s 879ms/step - loss: 8.3683 - acc: 0.0681
Epoch 2/180
283/283 [==============================] - 203s 719ms/step - loss: 7.9549 - acc: 0.0698
Epoch 3/180
283/283 [==============================] - 204s 719ms/step - loss: 7.7267 - acc: 0.0701
Epoch 4/180
283/283 [==============================] - 203s 718ms/step - loss: 7.4033 - acc: 0.0698
Epoch 5/180
283/283 [==============================] - 203s 719ms/step - loss: 7.1138 - acc: 0.0694
Epoch 6/180
283/283 [==============================] - 204s 720ms/step - loss: 6.9185 - acc: 0.0705
Epoch 7/180
283/283 [==============================] - 204s 719ms/step - loss: 6.6631 - acc: 0.0698
Epoch 8/180
283/283 [==============================] - 203s 719ms/step - loss: 6.4601 - acc: 0.0688
Epoch 9/180
283/283 [==============================] - 204s 720ms/step - loss: 6.3088 - acc: 0.0691
Epoch 10/180
283/283 [==============================] - 203s 719ms/step - loss: 6.1130 - acc: 0.0696

283/283 [==============================] - 204s 720ms/step - loss: 0.0420 - acc: 0.9888
Epoch 163/180
283/283 [==============================] - 204s 720ms/step - loss: 0.0260 - acc: 0.9923
Epoch 164/180
283/283 [==============================] - 204s 721ms/step - loss: 0.0210 - acc: 0.9947
Epoch 165/180
283/283 [==============================] - 204s 720ms/step - loss: 0.0136 - acc: 0.9953
Epoch 166/180
283/283 [==============================] - 217s 765ms/step - loss: 0.0094 - acc: 0.9964
Epoch 167/180
283/283 [==============================] - 207s 731ms/step - loss: 0.0114 - acc: 0.9962
Epoch 168/180
283/283 [==============================] - 204s 720ms/step - loss: 0.0251 - acc: 0.9922
Epoch 169/180
283/283 [==============================] - 204s 721ms/step - loss: 0.0415 - acc: 0.9873
Epoch 170/180
283/283 [==============================] - 204s 720ms/step - loss: 0.0387 - acc: 0.9888
Epoch 171/180
283/283 [==============================] - 204s 721ms/step - loss: 0.0168 - acc: 0

In [4]:
# loading test images
testPaths = list(paths.list_images('C:/Users/jdu12/Desktop/humpback/test/'))
print(len(testPaths))
test_images = np.array([get_image(img_path) for img_path in testPaths])
print(test_images.shape)

15610
(15610, 299, 299, 3)


In [15]:
# loading trained model
model_name = 'routine-130-0.99'
model = load_model('C:/Users/jdu12/Desktop/humpback/saved_model/InceptionResnetV2/' + model_name+ '.hdf5')

In [16]:
# predict!
pred = model.predict(test_images)

# testing and generating submission file
import warnings
from os.path import split

pred_dir = "C:/Users/jdu12/Desktop/humpback/prediction/InceptionResnetV2/"

with open(pred_dir + model_name + ".csv", "w") as f:
    f.write("Image,Id\n")
    top_5 = np.argsort(pred)[:,-1:-6:-1]   # get the top 5 most likely classes
    for i in range(top_5.shape[0]):
        cur_tags = ''
        cur_image_name = testPaths[i].split('/')[-1]
        for j in range(5):
            cur_tags = cur_tags + ' ' + class2id[top_5[i][j]]
        f.write("%s,%s\n" %(cur_image_name, cur_tags))     